In [22]:
from PIL import Image
import math
from collections import Counter
from xlwt import *
# grey = R * 0.3 + G * 0.59 + B * 0.11

In [25]:
def hist(inputfile, outputfile, typenum):
    im = Image.open(inputfile)
    width = im.size[0]
    height = im.size[1]
    rgb_im = im.convert('RGB')

    pixels  = []
    counts = []
    important = []
    for i in range(0, width):
        for j in range(0, height):
            r, g, b = rgb_im.getpixel((i, j))
            value = math.floor(r/32)*64 + math.floor(g/32)*8 + math.floor(b/32)
            if (value in pixels):
                counts[pixels.index(value)] += 1
            else:
                counts.append(1)
                pixels.append(value)
    for i in range(typenum + 1):
        position = counts.index(max(counts))
        important.append(pixels[position])
        counts.remove(max(counts))
        pixels.remove(pixels[position])

    exist = [0]*len(important)
    column = []
    for i in range(0, width):
        column = []
        for j in range(0, height):
            r, g, b = rgb_im.getpixel((i, j))
            value = math.floor(r/32)*64 + math.floor(g/32)*8 + math.floor(b/32)
            column.append(value)
        for k in range(len(important)):
            if important[k] in column:
                exist[k] += 1

    background = important[exist.index(max(exist))]

    important.remove(background)
    propotions = []
    for i in range(len(important)):
        propotions.append([])
    for i in range(0, width):
        column = []
        for j in range(0, height):
            r, g, b = rgb_im.getpixel((i, j))
            value = math.floor(r/32)*64 + math.floor(g/32)*8 + math.floor(b/32)
            column.append(value)
        for k in range(len(important)):
            propotions[k].append(column.count(important[k])/(column.count(important[k])+column.count(background)))
    #print(propotions)
    output = []
    for k in range(len(important)):
        output.append([])
        count = []
        for i in range(len(propotions[k])):
            if propotions[k][i] == 0:
                count.append(-1)
            else:
                vnum = 0
                for j in range(0, 10):
                    if i+j < len(propotions[k]):
                        if ((propotions[k][i+j] > propotions[k][i]*0.9) and (propotions[k][i+j] < propotions[k][i]*1.1)):
                            vnum += 1
                count.append(vnum)
        #print(count)
        new = 1
        for i in range(len(count)-3):
            if count[i] + count[i+1] + count[i+2] == -3:
                new = 1
            elif count[i] == max(count) and new == 1:
                new = 0
                output[k].append(sum(propotions[k][i:i + max(count)])/max(count))

    print(output)
    file = Workbook(encoding = 'utf-8')
    table = file.add_sheet('data')
    for k in range(len(important)):
        for i,p in enumerate(output[k]):
            table.write(k,i,p)
            print(k,i,p)
    file.save(outputfile)


In [26]:
hist('222.png', 'hist.xls', 2)

[[0.2172590740238407, 0.9624261213090908, 0.21973726199340496], [0.1889134757014304, 0.1908909829158048, 0.44799395467857134]]
0 0 0.2172590740238407
0 1 0.9624261213090908
0 2 0.21973726199340496
1 0 0.1889134757014304
1 1 0.1908909829158048
1 2 0.44799395467857134
